# Ringed Seal Detector

This notebook has the final code used to create the ringed seal detector. 

In [1]:
import pandas as pd
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.audio.audio_loader import AudioFrameLoader, AudioLoader, SelectionTableIterator
from ketos.audio.spectrogram import MagSpectrogram
from ketos.neural_networks.dev_utils.detection import batch_load_audio_file_data, filter_by_threshold
import numpy as np
import tensorflow as tf
from ketos.data_handling.data_feeding import BatchGenerator
from ketos.neural_networks.resnet import ResNetInterface
import shutil
from ketos.data_handling.data_feeding import JointBatchGen
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import csv
from sklearn.metrics import confusion_matrix as confusion_matrix_sklearn

import warnings
warnings.filterwarnings("ignore")

print('done')

C:\Users\kzammit\Miniconda3\envs\ketos_env\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


done


### Deploy Detector on Audio Data

In [ ]:
pp_folder = r'E:\baseline-w-normalization\pp'
temp_folder = pp_folder + '\\' + 'ringedS_tmp_folder'
detections_csv = pp_folder + '\\' + 'detections_raw_pearce_point_fine_tuned.csv'
threshold = 0
step_size = 1.0
batch_size = 16
buffer = 0.5 
model_for_pp = r'E:\baseline-w-normalization\pp\fine-tune\pearce-point-fine-tuned-model.kt'
spectro_file = r'E:\baseline-w-normalization\spec_config_100-1200Hz-0.032-hamm-adjustrange.json'

audio_folder = r'E:\baseline-w-normalization\pp\audio'

model = ResNetInterface.load(model_file=model_for_pp, new_model_folder=temp_folder)

audio_repr = load_audio_representation(path=spectro_file)

spec_config = audio_repr['spectrogram']

audio_loader = AudioFrameLoader(path=audio_folder, duration=spec_config['duration'],
                                    step=step_size, stop=False, representation=spec_config['type'],
                                    representation_params=spec_config, pad=False)
detections = pd.DataFrame()

batch_generator = batch_load_audio_file_data(loader=audio_loader, batch_size=batch_size)

for batch_data in batch_generator:
    # Run the model on the spectrogram data from the current batch
    batch_predictions = model.run_on_batch(batch_data['data'], return_raw_output=True)

    # Lets store our data in a dictionary
    raw_output = {'filename': batch_data['filename'], 'start': batch_data['start'], 'end': batch_data['end'],
                  'score': batch_predictions}

    batch_detections = filter_by_threshold(raw_output, threshold=threshold)

    detections = pd.concat([detections, batch_detections], ignore_index=True)

detections.to_csv(detections_csv, index=False)

  0%|                                                                                          | 0/299 [00:00<?, ?it/s]

In [5]:
# Convert to Raven

results_table = detections

cols = ['filename']
results_table.loc[:,cols] = results_table.loc[:,cols].ffill()
results_table['Selection'] = results_table.index +1
results_table['View'] = 'Spectrogram 1'
results_table['Channel'] = 1
results_table['Begin Path'] = audio_folder + '\\' + results_table.filename
results_table['File Offset (s)'] = results_table.start
results_table = results_table.rename(columns={"start": "Begin Time (s)", "end": "End Time (s)", "filename": "Begin File"})
results_table['Begin File'] = results_table['Begin File']
results_table['Low Freq (Hz)'] = 100
results_table['High Freq (Hz)'] = 1200

pos_results = results_table[results_table.label == 1]
neg_results = results_table[results_table.label == 0]

results_table.to_csv(main_folder + '\\' + 'raven_formatted_results.txt', index=False, sep='\t')
pos_results.to_csv(main_folder + '\\' + 'raven_formatted_pos_results.txt', index=False, sep='\t')